# 1. Setting Up

In [ ]:
import rdkit
print(rdkit.__version__)

import Auto3D
print(Auto3D.__version__)
import shutil
# import openbabel
# import yaml
# import torch

# 2. RDKit

## 2.1 Reading and writing molecules
RDKit can read a single molecule from a SMILES string, or a set of molecules from a file. The file format can be SDF, Mol, etc.

RDKit can write molecules into SMILES, or blocks of text with different formats. This include XYZ, SDF and Mol. 

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdmolfiles

In [ ]:
#Reading Molecule(s)
methel_benzene = Chem.MolFromSmiles('Cc1ccccc1')  #read from SMILES
methel_benzene

In [ ]:
mols = Chem.SmilesMolSupplier("files/DA.smi", titleLine=False)
for mol in mols:
    print(mol.GetProp("_Name"), mol.GetNumAtoms())

In [ ]:
mols = Chem.SDMolSupplier("files/DA.sdf")
for mol in mols:
    print(mol.GetProp("_Name"), mol.GetNumAtoms(), mol.GetProp("E_tot"))

In [ ]:
#Writing Molecules(s)
mols = Chem.SDMolSupplier("files/DA.sdf", removeHs=False)
xyz_blocks = []
for mol in mols:
    xyz_blocks.append(Chem.MolToXYZBlock(mol))
with open("files/DA.xyz", "w") as f:
    for block in xyz_blocks:
        f.write(block)

In [ ]:
mols = Chem.SDMolSupplier("files/DA.sdf")
for mol in mols:
    print(Chem.MolToSmiles(mol))

## 2.2 Visalize molecues

RDkit can visualize an individual molecule, a list of molecules or even molecles in a dataframe.

In [ ]:
from rdkit.Chem import Draw
import pandas as pd
from rdkit.Chem import PandasTools

In [ ]:
#Visualization Molecule(s)
mol

In [ ]:
#prepare a list of molecules
df = pd.read_csv("files/20_mols.csv")
df.head()

In [ ]:
smiles = [smi for smi in df["acids"]]
mols = [Chem.MolFromSmiles(smi) for smi in smiles]
Draw.MolsToGridImage(mols, molsPerRow=5)

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df, smilesCol="acids", molCol="acid_structure")
df.head()

## 2.3 Getting information from molecules
We could get basic atomic, bond-level or molecule level information from molecules.

In [ ]:
#Get Molecular & Atomic information
mol  = Chem.MolFromSmiles("C[C@@H](O)c1cccc(-c2ccc(OCC(=O)O)cc2)n1")
print("# of conformers: ", mol.GetNumConformers())
print("# of atoms: ", mol.GetNumAtoms())
print("# of bonds: ", mol.GetNumBonds())
print("# of heavy atoms: ", mol.GetNumHeavyAtoms())
print("# of aromatic atoms: ", list(mol.GetAromaticAtoms()))

In [ ]:
#get atomic information
atoms = mol.GetAtoms()
for atom in atoms:
    print(atom.GetSymbol(), atom.GetAtomicNum(), atom.GetIdx(), atom.GetTotalNumHs(), atom.GetFormalCharge(), atom.GetHybridization())

In [ ]:
# get bond information
bonds = mol.GetBonds()
for bond in bonds:
    print(bond.GetIdx(), bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond.GetBondType(), bond.GetIsAromatic())

## 2.4 Getting descriptors from molecules
RDkit can give us molecular fingerprints or descriptors, which are numerical molecular representation that many downstream modeling applications rely on.

In [ ]:
# Descriptors (fingerprint, similarity, other molecular properties) Phil
import numpy as np
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import AllChem

In [ ]:
print("molecular weight: %.2f" % (Descriptors.MolWt(mol)) )
print("TPSA: %s" % (Descriptors.TPSA(mol)))  #topological polar surface area
print("LogP: %.2f" % (Descriptors.MolLogP(mol)))

In [ ]:
#visualize atomic contribution into the LogP value
contribs = rdMolDescriptors._CalcCrippenContribs(mol)
img = SimilarityMaps.GetSimilarityMapFromWeights(mol, [x for x, y in contribs], colorMap="jet", contourLines=10)

In [ ]:
# calculate all descriptors
num_descriptors = [x[0] for x in Descriptors._descList]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(num_descriptors)
print(len(num_descriptors))

In [ ]:
all_descriptors = calc.CalcDescriptors(mol)
print(all_descriptors)

In [ ]:
#fingerprints
bi = {}
MFP = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024, bitInfo=bi)
print(MFP)

In [ ]:
MFP.ToBitString()

In [ ]:
MFP2 = np.array(MFP)
MFP2

In [ ]:
# meaning of the bits
mol

In [ ]:
MFP2_svg = Draw.DrawMorganBit(mol, 15, bi, useSVG=True)
MFP2_svg

In [ ]:
rdkbi = {}
rdkfp = Chem.RDKFingerprint(mol, maxPath=5, bitInfo=rdkbi,fpSize=512)
rdk_svg = Draw.DrawRDKitBit(mol, 1, rdkbi, useSVG=True)
rdk_svg

## 2.5 Similarity comparasion using fingerprints

In [ ]:
#similarity
mol  = Chem.MolFromSmiles("C[C@@H](O)c1cccc(-c2ccc(OCC(=O)O)cc2)n1")
MFP = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024, bitInfo=bi))
mol

In [ ]:
mol2  = Chem.MolFromSmiles("C[C@@H](O)c1cccc(-c2ccc(NCC(=O)OCCC)cc2)n1")
MFP2 = np.array(AllChem.GetMorganFingerprintAsBitVect(mol2, radius=2, nBits=1024, bitInfo=bi))
mol2

In [ ]:
similarity12 = round(np.dot(MFP, MFP2)/(np.linalg.norm(MFP)*np.linalg.norm(MFP2)), 3)
print(similarity12)

## 2.6 Scafold match

In [ ]:
#substracture search
df = pd.read_csv("files/20_mols.csv")
df.head()

smiles = [smi for smi in df["acids"]]
mols = [Chem.MolFromSmiles(smi) for smi in smiles]
Draw.MolsToGridImage(mols, molsPerRow=5)

In [ ]:
substract= Chem.MolFromSmarts('[*r6R1]1[cR2]2[cR1][cR1][cR1][cR1][cR2]2[*r6R1][*r6R1][*r6R1]1')
substract

In [ ]:
mols_subset = [mol for mol in mols if mol.HasSubstructMatch(substract)]
Draw.MolsToGridImage(mols_subset, molsPerRow=5)

# 3. Auto3D
[Auto3D](https://github.com/isayevlab/Auto3D_pkg) is a Python-based package for generating low-energy conformers from SMILES. It automatizes the stereoisomer enumeration and duplicate filtering process, 3D building process, fast geometry optimization and ranking process using ANI and AIMNet neural network atomistic potentials.

In [ ]:
from Auto3D.auto3D import options, main

## 3.1 Generate low-energy 3D structures with Auto3D

In [ ]:
if __name__ == "__main__":
    input_path = "files/DA.smi"
    args = options(input_path, k=1, use_gpu=False, verbose=True)   #args specify the parameters for Auto3D 
    out = main(args)            #main acceps the parameters and run Auto3D

## 3.2 Calculate thermodynamic properties with the 3D structures

In [ ]:
from Auto3D.ASE.thermo import calc_thermo

In [ ]:
out = "files/DA.sdf"

out_thermo = calc_thermo(out, "AIMNET", opt_tol=0.003)
print(out_thermo)  #enthalpy, entropy and Gibbs free energy are stored in the SDF file

Auto3D also provides wrapper function for single point energy calculation, geometry optimization, etc. More examples can be found in the GitHub site: https://github.com/isayevlab/Auto3D_pkg/tree/main/example

# 4. In-class Practice

In the following practice, we are going to calculate the electronic reaction energy for the following Diels-Alder reaction. The electronic reaction energy $\Delta E^{e}$ is defined as the difference between the electronic energy of the product ($E^e_{product}$) and the reactants ($E^e_{diene}$ and $E^e_{dieneophile}$):
$$\Delta E^{e} = E^e_{product} - E^e_{diene} - E^e_{dieneophile}  (1)$$
You are given a file (`files/DS.smi`) containing the SMILES for the reactants and product. To calculate the electronic reaction energy, you need to find the 3D structures and electronic energies for each of the SMILES. 

- *Hint1: Remember how we transformed SMILES into 3D structures in SDF file in section `3.1`;*
- *Hint2: Remember how we extract molecular property with RDKit in section `2`.*


![](practice.png)

**Bonus question**: can you calculate the reaction Gibbs free energy ($\Delta G$)? You already got the 3D structures and electronic energies in the first part of the practice. The reaction Gibbs free energy $\Delta G$ is defined as
$$\Delta G = G_{product} - G_{diene} - G_{dieneophile}  (2)$$
Remember that we could calculate Gibbs free energy for each molecule in section `3.2`.

# 5. Reference

- https://www.rdkit.org/docs/
- https://www.rdkit.org/docs/GettingStartedInPython.html
- https://www.rdkit.org/docs/Cookbook.html
- https://github.com/isayevlab/Auto3D_pkg
- https://github.com/isayevlab/Auto3D_pkg/tree/main/example
- https://doi.org/10.1021/acs.jcim.2c00817